# Bag-of-Objects TF-IDF

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pickle
import pandas as pd
from tqdm import tqdm
import os

from dataloader import he_sampling
from data import FlickrDataset, CocoDataset, VGDataset
from PIL import Image
import matplotlib.patches as patches

from sklearn.feature_extraction.text import TfidfTransformer


In [3]:
ds = VGDataset(sg_path=None)

In [5]:
l_test = ds.d_split['test']

In [6]:
l_train_terms = []
for img_id in ds.d_split['train']:
    sg = ds.imgid2sg(img_id)
    l_train_terms.append(sg['obj_labels'])
    
l_test_terms = []
for img_id in ds.d_split['test']:
    sg = ds.imgid2sg(img_id)
    l_test_terms.append(sg['obj_labels'])    

In [7]:
train_vocab = set([t for d in l_train_terms for t in d])

In [8]:
test_vocab = set([t for d in l_test_terms for t in d])

In [9]:
vocab = train_vocab.union(test_vocab)
vocab2idx = {v:i for i, v in enumerate(sorted(list(vocab)))}

In [10]:
train_mat = np.zeros((len(l_train_terms), len(vocab)))
test_mat = np.zeros((len(l_test_terms), len(vocab)))

In [11]:
for i_d, d in enumerate(l_train_terms):
    for t in d:
        idx = vocab2idx[t]
        train_mat[i_d, idx] += 1
        
for i_d, d in enumerate(l_test_terms):
    for t in d:
        idx = vocab2idx[t]
        test_mat[i_d, idx] += 1        

In [12]:
tfidf = TfidfTransformer()

In [13]:
tfidf.fit(train_mat)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [14]:
test_tfidf = tfidf.transform(test_mat).toarray()

In [15]:
sim_mat = test_tfidf.dot(test_tfidf.T)

In [16]:
result_dir = '/data/project/rw/viewer_CBIR/viewer/vg_coco_results/boo_tfidf'
for i_query, test_id in enumerate(tqdm(l_test)):  # query id
    l_sim = []
    for j_target, target_id in enumerate(l_test):
        l_sim.append(sim_mat[i_query, j_target])
    df = pd.DataFrame({'target_id': l_test, 'sim': l_sim}).sort_values('target_id')

    df[['target_id', 'sim']].to_csv(os.path.join(result_dir, f'{test_id}.tsv'), sep='\t', header=False, index=False)

100%|██████████| 13203/13203 [16:10<00:00, 13.60it/s]
